<a href="https://colab.research.google.com/github/ArifAygun/Sentiment-Analysis-SMRT/blob/main/SMRT_Statistical_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [57]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from google.colab import drive
drive.mount('/content/drive/')
%cd /content/drive/My Drive/Web_Scraping/SMRT/

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/My Drive/Web_Scraping/SMRT


In [58]:
reviews= pd.read_excel('Capterra_SMRT_Final.xlsx', 'Final')

In [59]:
reviews.head().transpose()

,0,1,2,3,4
Date,2019-08-20 00:00:00,2020-06-18 00:00:00,2020-08-10 00:00:00,2020-08-10 00:00:00,2020-08-10 00:00:00
User Name,Joshua V.,Daniel C.,Kyle M.,Jason Y.,Amy W.
User Position,CEO,Owner,Director of Operations,Vice President,Manager
Department,Information Technology and Services,Consumer Services,Consumer Services,Retail,Policies and ProceduresConsumer Services
Employees,1-10,11-50,11-50,51-200,11-50
Use Period,Less than 6 months,2+ years,1-2 years,2+ years,2+ years
Overall Rating,5,5,5,5,5
Ease of Use,5,5,5,5,4
Customer Service,5,5,5,5,5
Features,5,5,5,5,4


In [60]:
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115 entries, 0 to 114
Data columns (total 21 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   Date                     115 non-null    datetime64[ns]
 1   User Name                115 non-null    object        
 2   User Position            113 non-null    object        
 3   Department               115 non-null    object        
 4   Employees                115 non-null    object        
 5   Use Period               115 non-null    object        
 6   Overall Rating           115 non-null    int64         
 7   Ease of Use              115 non-null    int64         
 8   Customer Service         115 non-null    int64         
 9   Features                 115 non-null    int64         
 10  Value for Money          110 non-null    float64       
 11  Likelihood to Recommend  115 non-null    int64         
 12  Headline                 115 non-nul

## UNIQUE VALUES

In [61]:
unique_value_counts = reviews.nunique()

# Print the number of unique values for each column
print(unique_value_counts)

Date                        80
User Name                  113
User Position               26
Department                  14
Employees                    5
Use Period                   5
Overall Rating               3
Ease of Use                  5
Customer Service             5
Features                     4
Value for Money              3
Likelihood to Recommend      6
Headline                   112
Overall Rev                 94
Pros                       115
Cons                       115
Alternative Considered      12
Reasons for Choosing        53
Switched From               14
Reasons for Switching       80
Vendor Response              4
dtype: int64


### User Position

In [62]:
# Get the counts of each unique value in the "User Position" column
position_counts = reviews['User Position'].value_counts()

# Print the number of each unique value with its count
print("User Position Counts:")
print(position_counts)

User Position Counts:
Owner                                    52
President                                21
CEO                                       8
Vice President                            4
Manager                                   4
General Manager                           3
Managing Partner                          2
President/CEO                             1
Coordinator                               1
Store General Manager                     1
production manager                        1
Team Member Support Specialist            1
Customer Service Representative           1
Store Manager                             1
Chief Operations Officer                  1
CFO                                       1
Bookkeeper                                1
Admin assistant/manager                   1
Vice president                            1
President CEO                             1
Director of Operations                    1
Ceo                                       1
Customer C

### Department

In [ ]:
# Get the counts of each unique value in the "Department" column
department_counts = reviews['Department'].value_counts()

# Print the number of each unique value with its count
print("Department Counts:")
print(department_counts)